In [1]:
import pandas as pd
import numpy as np
import cv2
import glob # search for files that match a specific file pattern or name


from sklearn.model_selection import train_test_split

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
allImages = []
allLabels = []

input_shape = (64,64)

In [3]:
uninfected_images = 'C:/Users/rohit/Downloads/archive (10)/Malarial Cell Image Data/malarial_cell_image_data/cell_images/Uninfected/'
paracetized_images = "C:/Users/rohit/Downloads/archive (10)/Malarial Cell Image Data/malarial_cell_image_data/cell_images/Parasitized/"

paths = [uninfected_images , paracetized_images]

In [4]:
for path in paths:
    path2 = path + '/*.png'
    for file in glob.glob(path2):
        #print(file)
        
        
        # load the Images
        img = cv2.imread(file)
        
        if img is not None:
            resized = cv2.resize(img , input_shape , interpolation = cv2.INTER_AREA)
            allImages.append(resized)
            
            if path == uninfected_images:
                allLabels.append(1)
            else:
                allLabels.append(0)

In [5]:
allImages = np.array(allImages,dtype=np.float32)
allImages.shape

(27558, 64, 64, 3)

In [6]:
allLabels = np.array(allLabels, dtype=np.int32)
allLabels.shape

(27558,)

In [7]:
# show  first image

img = allImages[4]
label = allLabels[4]
print(label)

1


In [8]:
cv2.imshow('img' , img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [9]:
input_shape = (64,64,3)

In [10]:
# normalize the values between 0 and 1

allImagesforModel = allImages / 255.0

In [11]:
# splitting the Data
x_train,x_test , y_train , y_test = train_test_split(allImagesforModel , allLabels , test_size=0.2 , random_state=42)

In [12]:
print('shape of input train data : ',len(x_train))
print('shape of input test data : ',len(x_test))
print('shape of output train data : ',len(y_train))
print('shape of output test data : ',len(y_test))

shape of input train data :  22046
shape of input test data :  5512
shape of output train data :  22046
shape of output test data :  5512


In [13]:
x_train[1] , y_train[1]

(array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], dtype=float32),
 0)

### Building the CNN model

In [14]:
from keras.models import Sequential
from keras.layers import MaxPooling2D , Conv2D , Dense , Flatten

from keras.callbacks import ModelCheckpoint

In [15]:
model =Sequential()

model.add(Conv2D(input_shape=input_shape , filters=16 , kernel_size=(3,3) , padding='same' , activation='relu'))
model.add(Conv2D(filters=16 , kernel_size=(3,3) , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2) ,strides=(2,2)))


model.add(Conv2D(filters=32 , kernel_size=(3,3) , padding='same' , activation='relu'))
model.add(Conv2D(filters=32 , kernel_size=(3,3) , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2) ,strides=(2,2)))

model.add(Flatten())

model.add(Dense(1024 , activation='relu'))
model.add(Dense(1 , activation='sigmoid'))


model.compile(optimizer='adam' , loss = 'binary_crossentropy' , metrics=['accuracy'])
print(model.summary())




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 16)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                

In [16]:
batch = 32
epoch =10

stepPerEpoch = np.ceil(len(x_train)/batch)
ValidationStep = np.ceil(len(x_test)/batch)

print(stepPerEpoch , ValidationStep)

689.0 173.0


In [17]:
history = model.fit(x_train , y_train , batch_size=batch,
                   epochs=epoch , verbose=1 , validation_data=(x_test , y_test),
                   validation_steps=ValidationStep , steps_per_epoch=stepPerEpoch,
                   shuffle=True )

Epoch 1/10


689/689 [==============================] - 102s 141ms/step - loss: 0.3767 - accuracy: 0.7997 - val_loss: 0.1617 - val_accuracy: 0.9556
Epoch 2/10
689/689 [==============================] - 78s 113ms/step - loss: 0.1399 - accuracy: 0.9555 - val_loss: 0.1319 - val_accuracy: 0.9615
Epoch 3/10
689/689 [==============================] - 76s 110ms/step - loss: 0.1130 - accuracy: 0.9611 - val_loss: 0.1302 - val_accuracy: 0.9572
Epoch 4/10
689/689 [==============================] - 74s 108ms/step - loss: 0.0959 - accuracy: 0.9676 - val_loss: 0.1339 - val_accuracy: 0.9632
Epoch 5/10
689/689 [==============================] - 76s 110ms/step - loss: 0.0725 - accuracy: 0.9748 - val_loss: 0.1532 - val_accuracy: 0.9612
Epoch 6/10
689/689 [==============================] - 75s 109ms/step - loss: 0.0547 - accuracy: 0.9813 - val_loss: 0.1887 - val_accuracy: 0.9592
Epoch 7/10
689/689 [==============================] - 75s 109ms/step - loss: 0.0345 - accuracy: 0.9892 - val_loss: 0.2982 - val

In [41]:
loss, accuracy = model.evaluate(np.array(x_test), np.array(y_test))

173/173 [==============================] - 4s 20ms/step - loss: 0.4167 - accuracy: 0.9512


In [43]:
accuracy

0.9511973857879639

In [38]:
def prepareImage(img):
    resized = cv2.resize(img , input_shape , interpolation = cv2.INTER_AREA)
    imgResult = np.expand_dims(resized , axis=0)
    imgResultult = imgResult/255.0
    
    return imgResult

def classify(inp_image):
    img = cv2.imread(inp_image)
    image_model = prepareImage(img)
    result = model.predict(image_model , verbose=1)
    if result[0][0]==1.0:
        print('Uninfected')
    else:
        print('infected')
        
        
classify("C:/Users/rohit/Data science/Major Project/Images/C38P3thinF_original_IMG_20150621_112043_cell_202.png")

1/1 [==============================] - 1s 1s/step
infected


In [39]:
model.save('model.h5')

C:\Users\rohit\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
